Задание. Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

Условия по варианту (ИУ5-62Б Вар.17)
ИУ5-62Б, ИУ5Ц-82Б Метод опорных векторов Случайный лес

17. https://www.kaggle.com/mathan/fifa-2018-match-statistics

**Загрузка датасета**

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Загрузка данных
df = pd.read_csv('FIFA 2018 Statistics.csv')
print("Первые 5 строк данных:\n", df.head())
print("\nИнформация о данных:\n", df.info())
print("\nПропуски в данных:\n", df.isnull().sum())

Первые 5 строк данных:
          Date          Team      Opponent  Goal Scored  Ball Possession %  \
0  14-06-2018        Russia  Saudi Arabia            5                 40   
1  14-06-2018  Saudi Arabia        Russia            0                 60   
2  15-06-2018         Egypt       Uruguay            0                 43   
3  15-06-2018       Uruguay         Egypt            1                 57   
4  15-06-2018       Morocco          Iran            0                 64   

   Attempts  On-Target  Off-Target  Blocked  Corners  ...  Yellow Card  \
0        13          7           3        3        6  ...            0   
1         6          0           3        3        2  ...            0   
2         8          3           3        2        0  ...            2   
3        14          4           6        4        5  ...            0   
4        13          3           6        4        5  ...            1   

   Yellow & Red  Red  Man of the Match  1st Goal        Round  PSO  

**Обработка пропусков**

In [25]:
# Заполнение пропусков
df.fillna({
    'Own goals': 0,
    'Own goal Time': 0,
    '1st Goal': df['1st Goal'].median(),
    'Corners': df['Corners'].median()
}, inplace=True)


**Удаление или преобразование нечисловых столбцов**

In [26]:
# Удаляем столбцы, которые не могут быть преобразованы в числа (например, даты)
df = df.drop(['Date'], axis=1)  # Удаляем столбец с датой

# Label Encoding для категориальных признаков (если они остались)
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    if col != 'Man of the Match':  # Целевая переменная
        df[col] = LabelEncoder().fit_transform(df[col])

# One-Hot Encoding для целевой переменной (если нужно)
df['Man of the Match'] = LabelEncoder().fit_transform(df['Man of the Match'])

**Масштабирование числовых признаков**

In [27]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('Man of the Match')  # Исключаем целевую переменную

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

**Разделение данных на обучающую и тестовую выборки**

In [28]:
X = df.drop('Man of the Match', axis=1)
y = df['Man of the Match']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Обучение моделей**

In [29]:
# SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

**Оценка качества моделей**

In [30]:
# Метрики для SVM
print("\nSVM Metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm))
print("Recall:", recall_score(y_test, y_pred_svm))
print("F1-Score:", f1_score(y_test, y_pred_svm))

# Метрики для Random Forest
print("\nRandom Forest Metrics:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1-Score:", f1_score(y_test, y_pred_rf))


SVM Metrics:
Accuracy: 0.5128205128205128
Precision: 0.6666666666666666
Recall: 0.4166666666666667
F1-Score: 0.5128205128205128

Random Forest Metrics:
Accuracy: 0.6666666666666666
Precision: 0.9230769230769231
Recall: 0.5
F1-Score: 0.6486486486486487
